In [1]:
# SELECT DISTINCT ?personVal ?relationVal ?toPVal WHERE { 
#     ?s a tbio:Person .
#     ?s ?p ?o .

#     ?p rdfs:subPropertyOf ?familyOP .
#     FILTER ( ?familyOP = tbio:hasFamilyRelation || ?familyOP = tbio:isFamilyRelationOf ) .

#     BIND(STR(?s) AS ?personStr) .
#     BIND(REPLACE(?personStr, "http://tbio.orient.cas.cz#", "") AS ?personVal) .
#     BIND(STR(?p) AS ?relationStr) .
#     BIND(REPLACE(?relationStr, "http://tbio.orient.cas.cz#", "") AS ?relationVal) .
#     BIND(STR(?o) AS ?toPStr) .
#     BIND(REPLACE(?toPStr, "http://tbio.orient.cas.cz#", "") AS ?toPVal) .
# }

In [2]:
import pandas as pd

In [3]:
filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/20190520'

filename = 'familyRelations_20190516.tsv'

read_filename = '{0}/{1}'.format(filepath, filename)

In [4]:
workDf = pd.read_csv(read_filename, delimiter='\t')
workDf = workDf.fillna('')
workDf.shape, workDf.head()

((34141, 3),   ?personVal ?relationVal    ?toPVal
 0        丁瑞彬   isFatherOf    丁瑞彬的女子1
 1        李文後   isFatherOf    李文後的男子3
 2        李文後   isFatherOf    李文後的男子2
 3        林劍清   isFatherOf        林希頂
 4        林玉雕   isFatherOf  林玉雕的男、女子8)

# Assign all relations

In [5]:
workList = []
for idx in range(0, len(workDf)):
    row = workDf.loc[idx]

    sPerson = str(row['?personVal'])
    relation = str(row['?relationVal'])
    tPerson = str(row['?toPVal'])
    
    if sPerson == '' or tPerson == '':
        continue
        
    toRow = [sPerson, relation, tPerson]

    if toRow not in workList:
        workList.append(toRow)
    else:
        print("DUP!", toRow)
    
print(len(workList))

34129


# Filter out inverse relations

In [6]:
def findInverse(INsPerson, INtPerson):
    for work in workList:
        sPerson = work[0]
        relation = work[1]
        tPerson = work[2]
        
#         if INsPerson == sPerson and INtPerson == tPerson:
# #             print(INsPerson, INtPerson, work)
#             return relation
        if INsPerson == tPerson and INtPerson == sPerson:
#             print(INsPerson, INtPerson, work)
            return relation
    return ''

relationList = []
for work in workList:
    sPerson = work[0]
    relation = work[1]
    tPerson = work[2]

    if 'has' in relation:
        relationList.append(work)
        continue
    
    OUTrelation = findInverse(sPerson, tPerson)
    if OUTrelation == '':
        relationList.append(work)
        
#     if 'has' in OUTrelation:
#         resList.append([sPerson, OUTrelation, tPerson])
#     elif OUTrelation != '':
#         resList.append(work)

print(len(relationList), relationList[0:5])

31684 [['辜顯榮', 'isFatherOf', '原配__辜顯榮長女'], ['吳運豐', 'hasFather', '吳鴻森'], ['林自炎', 'hasFather', '林歡邦'], ['吳芳英', 'hasFather', '吳鴻麟'], ['許碧瑜', 'hasFather', '許丙']]


In [7]:
# orgDf = pd.DataFrame(resList, columns=['SPerson', 'Relation', 'TPerson'])
# orgDf.drop_duplicates(keep='first', inplace=True)
# orgDf.sort_values(by=['SPerson', 'TPerson'], inplace=True)
# orgDf.head()

In [8]:
# orgDf.count()

# Read person-family map table

In [9]:
familymembers = 'Familymembers_20190521.xlsx'

read_familymembers = '{0}/{1}'.format(filepath, familymembers)

In [10]:
fmDf = pd.read_excel(read_familymembers)
fmDf.shape, fmDf.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/20190520/Familymembers_20190521.xlsx'

In [ ]:
fmDic = {}
# familiesDic = []
for idx in range(0, len(fmDf)):
    row = fmDf.loc[idx]

    person = str(row['personStr'])
    family = str(row['familyStr'])
    
    if person not in fmDic:
        fmDic[person] = family
    elif family != fmDic[person]:
        print("Dup:", person, family, fmDic[person])
        
#     if family not in familiesDic:
#         familiesDic.append(family)

# fmDic
# print(len(familiesDic), familiesDic)

In [ ]:
def getFamilyName(INperson):
    if INperson not in fmDic:
        return ''
    return fmDic[INperson]

relationFamilyList = []
for row in relationList:
    sPerson = row[0]
    relation = row[1]
    tPerson = row[2]

    sFamily = getFamilyName(sPerson)
    if sFamily == '':
        continue
        
    tFamily = getFamilyName(tPerson)
    if tFamily == '':
        continue

    if sFamily != tFamily:
        relationFamilyList.append([sPerson, relation, tPerson, sFamily, tFamily])
    
print(len(relationFamilyList), relationFamilyList[0:5])

# Remove parallel relations

In [ ]:
# marriageList = []
resList = []

def duplicate(INsPerson, INtPerson):
#     for row in marriageList:
    for row in resList:
        sPerson = row[0]
        tPerson = row[2]
        
        if sPerson == INsPerson and INtPerson == tPerson:
#             print(INsPerson, INtPerson)
            return True

        if tPerson == INsPerson and INtPerson == sPerson:
#             print(INsPerson, INtPerson)
            return True
    return False
            
for row in relationFamilyList:
    sPerson = row[0]
    tPerson = row[2]
    
    if duplicate(sPerson, tPerson) == False:
        resList.append(row)
#         marriageList.append([sPerson, tPerson])
    

In [ ]:
personList = []
familyList = []

for res in resList:
    sPerson = res[0]
#     relation = res[1]
    tPerson = res[2]
    sFamily = res[3]
    tFamily = res[4]
    
    if sPerson not in personList:
        personList.append(sPerson)
    
    if tPerson not in personList:
        personList.append(tPerson)
        
    if sFamily not in familyList:
        familyList.append(sFamily)
        
    if tFamily not in familyList:
        familyList.append(tFamily)
        
print(len(personList), personList[0:3])
print(len(familyList), familyList[0:3])

In [ ]:
personDf = pd.DataFrame(personList)
personDf = personDf.rename(columns = {0:'PersonName'})
personDf.sort_values(by=['PersonName'], inplace=True)
personDf.reset_index(inplace=True, drop=True)
personDf['ID'] = personDf.index + 1000
personDf.head()

In [ ]:
familyDf = pd.DataFrame(familyList)
familyDf = familyDf.rename(columns = {0:'FamilyName'})
familyDf.sort_values(by=['FamilyName'], inplace=True)
familyDf.reset_index(inplace=True, drop=True)
familyDf['ID'] = familyDf.index + 2000
familyDf.head()

# map person/family to ids

In [ ]:
def getPersonID(INperson):
    return personDf.loc[personDf['PersonName'] == INperson]['ID'].item()

def getFamilyID(INfamily):
    return familyDf.loc[familyDf['FamilyName'] == INfamily]['ID'].item()
    
finalList = []
for res in resList:
    sPerson = res[0]
    relation = res[1]
    tPerson = res[2]
    sFamily = res[3]
    tFamily = res[4]
    
    sPersonID = getPersonID(sPerson)
    tPersonID = getPersonID(tPerson)
    sFamilyID = getFamilyID(sFamily)
    tFamilyID = getFamilyID(tFamily)

    row = [sPersonID, tPersonID, sFamilyID, tFamilyID, 'Undirected', sPerson, tPerson, sFamily, tFamily, relation]
    if row not in finalList:
        finalList.append(row)

print(len(finalList), finalList[0:5])

In [ ]:
finalDf = pd.DataFrame(finalList, columns=['SourcePersonID', 'TargetPersonID', 'SourceFamilyID', 
                                           'TargetFamilyID', 'Type', 'SourcePersonName',
                                           'TargetPersonName', 'SourceFamilyName', 'TargetFamilyName',
                                           'Relation'])
finalDf.head()

In [ ]:
write_file_to = '{0}/{1}'.format(filepath, 'familyRelations_20190521_v6.xlsx')
finalDf.to_excel(write_file_to, index=False)

# Write Person/Family node IDs

In [ ]:
write_file_to = '{0}/{1}'.format(filepath, 'familyRelations_Node_20190521_v6.xlsx')
with pd.ExcelWriter(write_file_to) as writer:
    personDf.to_excel(writer, 'PersonNodes', index=False)
    familyDf.to_excel(writer, 'FamilyNodes', index=False)
    
    writer.save()